In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import gc
import re
import string
import operator
from collections import defaultdict
import seaborn as sns

import plotly.graph_objects as go
from plotly.subplots import make_subplots
from nltk.tokenize import word_tokenize
from sklearn.svm import SVC


from sklearn.linear_model import Ridge, Lasso, LinearRegression
from sklearn.model_selection import GridSearchCV, KFold, train_test_split, TimeSeriesSplit, StratifiedKFold, RandomizedSearchCV
from sklearn.utils import shuffle
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler, LabelEncoder, MinMaxScaler, StandardScaler
from sklearn.metrics import confusion_matrix, f1_score, mean_absolute_error, make_scorer
import lightgbm as lgb
import xgboost as xgb
import catboost as cb
import statsmodels.api as sm
from sklearn.metrics import make_scorer
from statsmodels.regression.linear_model import OLS
from datetime import datetime
from sklearn.model_selection import cross_val_score
from sklearn.cluster import DBSCAN
import shap
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
import re
# import tokenization
from wordcloud import STOPWORDS

from sklearn.model_selection import StratifiedKFold, StratifiedShuffleSplit
from sklearn.metrics import precision_score, recall_score, f1_score

import tensorflow as tf
import tensorflow_hub as hub
from tensorflow import keras
from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras.layers import Dense, Input, Dropout, GlobalAveragePooling1D
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, Callback

SEED = 1337

from nltk.corpus import stopwords

stop=set(stopwords.words('english'))


# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

/kaggle/input/train.csv
/kaggle/input/sample_submission_wgBqZCk.csv
/kaggle/input/test.csv


In [ ]:
!pip install pandas==0.25.1

In [ ]:
!wget --quiet https://raw.githubusercontent.com/tensorflow/models/master/official/nlp/bert/tokenization.py

In [ ]:
import tokenization

In [131]:
train = pd.read_csv('/kaggle/input/train.csv')
test = pd.read_csv('/kaggle/input/test.csv')
submission = pd.read_csv('/kaggle/input/sample_submission_wgBqZCk.csv')

In [132]:
train = train[~train.user_review.str.contains("#NAME?")]

In [133]:
train['user_review'] = train['user_review'].apply(lambda x: re.sub(r"([a-z\.!?])([A-Z])", r"\1 \2", x))
test['user_review'] = test['user_review'].apply(lambda x: re.sub(r"([a-z\.!?])([A-Z])", r"\1 \2", x))

In [134]:
def remove_url(text):
    url = re.compile(r'https?://\S+|www\.\S+')
    return url.sub(r'', text)

train['user_review'] = train['user_review'].apply(lambda x: remove_url(x))
test['user_review'] = test['user_review'].apply(lambda x: remove_url(x))

# let's remove html tags

def remove_html(text):
    html = re.compile(r'<.?>')
    return html.sub(r'', text)

train['user_review'] = train['user_review'].apply(lambda x: remove_html(x))
test['user_review'] = test['user_review'].apply(lambda x: remove_html(x))

In [135]:
# drop train rows with ambigious values and languages
train.drop([190, 355, 438, 920, 1672, 1686, 1729, 1829, 1863, 1899, 2016, 2217, 2528, 3209, 3459, 3704, 3733, 3869, 3976, 4035, 4075, 4838, 5481, 5506, 5587, 5860, 6041, 6430, 7189, 7724, 7931, 8232, 8527, 8926, 9196, 9209, 9261, 10339, 10345, 11353, 11511, 12014, 12887, 13271, 13300, 13607, 13632, 13905, 14026, 14409, 15258, 15513, 16614, 16823, 16889, 17093, 17093, 17370], inplace = True)

In [136]:
train.drop([5365], inplace = True)

In [137]:
# train pictograms
train.drop([128, 177, 193, 215, 2009, 2151, 3662, 3917, 8681, 9515, 12868, 13742, 13978, 14021, 14087, 14139, 14166, 14167, 14281, 14342, 14880, 15302,15955, 16012, 16568, 16757, 16916], inplace = True)

In [138]:
# change values in test 
test['user_review'][7] = "We all definitely played one of the Counter Strike games when we were little, there are modes such as Gun Game, Competitive etc. in this renewed version of Counter. The main ones are the bombing or killing all members of the other team in the competitive mode, but in the end You can give nice gifts, and you can team up with your friends and compete against others in the competition.The game's market can also be valuable, for example, a weapon I won from the competitor now corresponds to $ 5 on the market of CS GO. If you are lucky, you can enjoy the game by using some of the weapons that come in. In the race, you are moving towards the knife using all the weapons in the game and you have to kill one person with each weapon given, when you kill a person with that weapon, you switch to the other weapon and until this knife you go to the knife and you complete the mod when you last killed it. Fun. Better than 1.6."
test['user_review'][11] = "CS: GO' <= This is a really weird game ... So that's it. At the beginning of the game, you'll be automatically assigned to one of two teams: terrorists or special police. If you're lucky, there's one Russian in your team. Otherwise, know that you will face a team of 2-3 Russians! Well ... the main task of the terrorists is very simple: carry the explosives as if on their way, and in the place called 'Dust Two' you will find wooden boxes with wooden textures, but In a place called Hell, the fish in the fountain must be blown up and killed The main goal of the police is to go before the end of the hour, detach the ball and save the boxes, as well as the fish. Also, in places like 'Office' and 'Attack', there are slightly different rules. You just call the guys in yellow pajamas. All you have to do is carry it from point A or B to your opponent's side without being seen by the opposing team, bring it to their place of birth and draw it behind the yellow line! Then you will win! Even the captives will be saved! Bravo! And you don't have the right to play badly. If you start to play badly, your team will call you a stubborn husband or a lost animal. But if you want to fight with them, you have to call your mother! And don't be embarrassed to start voting to kick you out from a young age! However, a more civilized person might call you an adopted child or a baby found in a barrel. Oh, you can't play as well as you should. If you start playing well, your opponents will start calling you a cheater. And if you want to quarrel with them, you will also find out the secrets of your mother. Then they will stand firm in their decision that you are a fraud and start threatening to send a message to TsORGO. Even if the last survivor defeats the enemy and wins the title, if the team can't get the strong AWP gun they want, they will be reprimanded and blindfolded: / TL, DR: In short, you 'CS: GO' By playing, [+] learn Russian for free! [+] Learn to scold, learn to sing from people! [+] Discover all kinds of secrets about your mother! [+] Furthermore, your father was gay, and he was a naughty boy who found himself in a barrel. You'll learn a lot of interesting facts, etc. It's really amazing are you kidding"
test['user_review'][49] =  "Great gameplay. Pleasure from each completed game. Qualitative selection of players. Qualitative servers. Friendly and adequate community that will always cover you in the midst of battle and give good advice on the game. You won't see all of this here."
test['user_review'][53] = "It is a recommendation for people who are not content with the current, aim to continuous improvement and development, and use technology effectively. You can sell the items you drop in the game at the end of the current Drop item in the Community market. With this game, you can earn steam dollars to your Account and this allows you to buy new games for yourself with the accumulated dollars. The gameplay of the game is very good, the feeling of strokes is getting better day by day and there is no problem in the updates. Global Offensive is really good now leave 1.6 friends, the world has gone into this game, thank you for your interest."
test['user_review'][88] = "CS: GO - This: -Cases where nikerta does not drop out. -Knife skins are more expensive than your salary.-Absolutely non-heavenly system of ranks. -BANS to the right and left, whether you want it or not. -Cases are cheaper than the keys to them.-Cheaters are everywhere, VAC is resting. -This is when you kick at 15th round, being the 1st in the list by efficiency. -This is when you have played more than 400 fights in competitive mode, because You don’t try to take a chance and try to play on other maps because of the great new rank system. -This is when you have not played for several months and you have been lowered from 2 Kalash. This is when you pinch, and the tick rate does not count most bullets. The game became closer to Coll of duty-free, after the latest updates from Valve there is no spirit of competition, the mku was fixed first, I was silent, the speed of movement with the avapa was silent, the treads were like some stable in the stables, all the pistols were silent, well, let's say, silent revolver + anal plugs are already bust, I’m deleting the whiff. servers in China IN CHINA !!!!!!!!!!!!!!!!!!, B.L. Ya_T ,, there the word e-sports has never been heard-delete HaXYyGame degrades, discounts -75% every 3 months , new cheaters that just do not ban the cunning Gabenupatrul. I DO NOT RECOMMEND"
test['user_review'][302] = "So, what is CS GO? - CS GO is when weapons are more expensive than cool game devices. - CS GO is when you have to drop your beautiful Kalash to the whole team. - CS GO is when you open 20 chests and get 20 pieces of blue g ** on .- CS GO is when schoolchildren add you and offer over 9000 chests in exchange for your knife .- CS GO is when a schoolboy adds you and starts to beg for a gift of clothes worth 3500 rubles, while citing the argument yes you rich, what is it for you!. CS GO is when you receive links to Trojans 10 times a day in the hope of calculating stitch your inventory. - CS GO is when you are kicked out only because you are Russian. - CS GO is when you burn your chair after playing in competitive mode. - CS GO is when the “globals” create 10 accounts and get satisfaction by playing against “Kalash” .- CS GO is when you are Russian Noob, if you stayed against five, you killed four but did not finish off fifth .- CS GO is when you are Russian Noob with a score of 21-7-10, and the nearest teammate has 8-0-16 .- CS GO is when you learned Polish and Swedish in one or two skating rinks .- CS GO is when there are more Poles in competitive mode than in Poland itself .- CS GO is yes you open A alone, and the bomb is already about *** on B.- CS GO is when your stfu Russian answers to your advice and questions .- CS GO is when about *** l three ranks in a day only due to the fact that every time you play against the “globals” playing from the second account.- CS GO is when every third teammate is an assaulted silver.- CS GO is when five ♥♥♥♥♥♥♥♥♥♥ plays against you o with WUA. - CS GO is when you throw one “golden eagle” and four “Kalash” against five “suprims.” - CS GO is when you land all 30 rounds in one ♥♥♥♥♥♥♥♥♥♥ and you apply 99 in 7, and that one is random in - an arrow hits your head. - CS GO is when VAC exponentially bans professional players, but cannot ban ♥♥♥♥♥♥♥♥♥♥ but who shoots the whole team through three walls, while having 2.2 hours in the game. this is CS GO! 10 satisfied Gabes out of 10."
test['user_review'][341] = "Bad English for the world;) What remains to say to the game still a lot was said .... I write now my own comment and no Russian speaking I still cannot. Would you like to get to know or meet Russians? You like it to play a multiplayer 5 vs. 5 virtually to play alone. You\'ll love it in a communication/ teamplay- game to speak to oneself, because no one else does. It\'s fun when everyone does what he wants, all run in different directions without meaning and purpose. You\'re also of the opinion that expensive pro-gamer headsets are unnecessary. Who needs headphones or what to make announcements. You\'re tired of being scolded by the French in Call of Duty. You like to put more money in a game than necessary. You like it when you buy expensive weapons skins which are later less worth. You love to play a game for all eternity, But still not reach a new level as in other games. You like to be scolded as a noob because you have 2 kills less than the one who insults you. If you play a different weapon than all the others you are badly made. Your love for Eastern countries increases thereby you will harder. If you play alone then you meet strangers who can not speak your language or English. You are kicked out of the game when your team players do not like you. Your teammates are not always better than you, with the time you start Votekick like a Russian. Unnecessary games where you do not meet trollers / hackers does not interest you. So do not waste time with Face IT and other things .. You do not like beginner-friendly games because you are the PRO-Gamer. The game is aimed at competition with teamplay, announcements and hearing That is why it is so popular because it requires players, but not the pc. Since I play the game, I always look exactly who is adding me. In doubt, take no one suspect in your friendlist most want partial only your skins. I am Dimitri I send to you friendship request you accept! And now lie down quietly on the floor, click on LINK and give me your CS: GO Skins. Many thanks for your cooperationYou do not like long instructions and do not know what to do now. Then it\'s time to dive into the nicest gaming community there is. Did I say you can buy it?  I said you have to buy it! CS: GO is like a greedy woman, but you love her forever until the end of time. The feeling of hate and love connects and you can not change anything."
test['user_review'][343] = "A game full of cheats. After the Russians, whatever game I entered, like the Germans, this kind of tricks started to come up. I share a sample demo link. Please report anyone you think is cheating, do not allow this kind of dishonest person. Do not let anyone play with their own wrist on their own right .. CKY9LC0Although I have reported this guy on the man's profile, although I have shared this video on the man's profile, the man still plays the game and eats the right of the players who play with his wrist, even though I have complained to the steam executives from his profile. Think twice before buying this game"
test['user_review'][439] = "Advice not to buy"
test['user_review'][601] = "Language Content Simplified Chinese. Traditional Chinese. English Game is fun, but there's only one China in the world."
test['user_review'][653] = "Good game! I used to play WOT before, but I like the sea more, so I moved to World of Warships. A lot of action, a huge variety of ships, nice graphics - everything you need for a great game! I recommend you play! You’ll be 100% addicted and you won’t be able to play!"
test['user_review'][654] = "Good game! I used to play WOT before, but I like the sea more, so I moved to World of Warships. A lot of action, a huge variety of ships, nice graphics - everything you need for a great game! I recommend you play! You’ll be 100% addicted and you won’t be able to play!"
test['user_review'][728] = "Over 200h plaied in this game and mm doesnt got better. Wargaming ignores sugestions of community and dont fix the problem. You gona play vs 2 Tier higher ships, where you can do nothing, you under guned and under armored and cant run away... Balance is de facto an illusion and the hole game becomes a big grind story after a few hours. Save your time and dont try it."
test['user_review'][757] = "Great game — Good graphics — Short fights, not long ones like in tanks — Constant events that give out all kinds of buns — No art, which is a plus, like in tanks — It's Free to play — You can easily play without donation"
test['user_review'][1431] = "Early Access Review Spend your fucking nerves, soloing is easy fucking if you have no friends, the best way out is to hang yourself. Remember, the crayfish will play for you in this game, and the fucking cybersportsmen will play against you."
test['user_review'][1535] = "Good game. I do recommend"
test['user_review'][1725] = "Early Access Review A game with a lot of potential, a lot of strategy is necessary for the game to flow for your team. It would be extremely important that the creators of the game, put a means of communication through the voice, that would help a lot during the game"
test['user_review'][1802] = "Early Access Review When I think of it, it comes to my mind my grandpa plays this in the trench cetres> first and story of his experiences ... but fuck me without me grinding [triggered it] so I jump and eat the modem cables so i break this lcd tv and then i find this is a game for normal people so i go into the dark to hide from myself while playing yasuo via radio via MLG frula."
test['user_review'][1897] = "Early Access Review Fun game for casual gambling. Fast, action-packed rounds. I found some problems, it's still in beta, but none of them were during a match."
test['user_review'][2457] = "Moments of hellish lag, without a tutorial, you enter the game and you are completely lost, until you get someone to help you and understand the beginning. so far I don't know if there is a market or not, I can't speak in general why you need an item. very poorly managed. BLCK TRADE OF ACC NOVA, 5 DAYS FOR YOU TO MAKE ANY TRADE, YOU MAKE A CLASS THAT DEPENDS ON YOU TO GO TO THE MARKET BUY A LETTER AND ...? BLOCK. SERIOUSLY DO NOT WASTE TIME WITH THIS CRAP. Well, let's give it a new chance, then I was sure it’s not worth it. Grind clueless classes, but we’ll continue with people playing it shouldn’t be so bad, we’ll open this new class’s mission, 5 missions, these 5 are divided into 2 each, that is 10. take 100 of that item and 130 of those, the first 100 was 2 hours uninterrupted, with a low drop chance but takes out an average mob amount. second more 130 items, 3 mobs on the huge map and you have the chance drop of 30 for 1, 1 hour and dropei 2 items. IT'S VERY IDIOT TO DO DROP BASED ON LUCK, DO THE MISSAO BATE 500 DOS BIXO, BUT DO NOT DO THE DROP BASED ON LUCK, THAT ♥♥♥♥♥♥♥ SEEMS COORDINATED BY THE OF A 10 YEAR OLD BRAT. SERIOUSLY DO NOT WASTE TIME, THIS GAME WILL MAKE YOU ANGRY, TIBIA IS BETTER THAN THAT."
test['user_review'][2955] = "Excellent game. Bad administration. Lagged servers, locked channels, characters stuck in limbo. Zero staff pronouncement. Excellent game. Bad administration. Lagged servers, locked channels, characters stuck in limbo. Zero staff pronouncement."
test['user_review'][3275] = "Early Access Review I thought game well, a 3D moba, it reminded me a lot of Smite, I played very little but I intend to play for longer !. Negative Points: At least with me the video settings were very strange, with drops in fps, but I think it was because of my video drivers. I was very disgusted because I bought the game on sale, 2 days passed and there was a giveaway of the game and many got it for free:"
test['user_review'][3397] = "Early Access Review I need Chinese I need Chinese I need Chinese I need Chinese"
test['user_review'][4144] = "Early Access Reviewgame sapper! mega fishing simulator but she slightly agarichut me. you need to make it so that the Mesuri River is free. I played and the money ran out and had to be deleted and downloaded again."
test['user_review'][4618] = "Good game. Recommend"
test['user_review'][4998] = "Early Access Review The best game I’ve played lately. Fire gameplay is just 10 out of 10. Imagine DotA 2 in the MMO style and without Tim downs and get it. ATTENTION! It is addictive."
test['user_review'][5037] = "Product received for free Early Access Review And so the second part of the dungeon defense has become places both better and worse. In general, I liked it and still like it. Better: + Well, all the same, they added more heroes and the ability to customize them. + Added more servers. + Made it free. + Improved the graphics a bit. + Made the levels shorter for each other + Made more or less the balance of the heroes. Worse: - Well, it's donat , it was still there. - Servers lag. - Added the hero’s IMBA. - The graphics may be better, but it’s not optimized."
test['user_review'][5217] = "Early Access Review Dungeon Defenders 1 Top! Dungeon Defenders Eternity Flop! Dungeon Defenders II Top³! Dungeon Defenders II is a nice game for tower defense lovers! There are still some bugs atm, and much is not finished yet .. But Guys! Its a alpha! i get eye cancer by reading this mimimi Comments! Waits and enjoying this Free game! For those who speak even less English than I do: P. Stop the negative comments! It's an alpha and it's now playable for free. So before you post senseless comments about how shitty something is, you'd better write them on their forums where developers can read it and worry! I can't see a review from someone who doesn't even have 10 hours of play. As already written above eye cancer Trendy Entertainment Keep it up! Spelling mistakes are special effects of my keyboard and are only for general amusement!"
test['user_review'][5413] = "The game is good, BUT DO IT SO IT CAN BE ADDED TO FRIENDS THROUGH THE STIM GAME !!!!!!!!!! The game is good, BUT I like THAT you CAN ADD FRIENDS THROUGH the GAME IN STEAM !!!!!!!!!!"
test['user_review'][5534] = "Everyone can run away or give up for a moment, so let's keep walking There's something that only you can do Protect your dream! I can be an important friend and become a strong self. I have power that I don't know. \ U3000 If a heart ignites, no wish is a lie. It will surely come true. Sometimes it's not a sunny day, but sometimes it rains cold, but I'll open my umbrella. There's no map in my life, but it's \ u3000, so I'm free to go anywhere, you may! Aim faster than the wind! Far from the sky, I can meet my new self. The courage I don't know sleeps. \ U3000 When I notice the heart, the sucking in my chest will stop. Grab a dazzling tomorrow and let your loved ones become your strong self! Break the bear you! The colliding wall becomes a hot heartbeat weapon."
test['user_review'][5832] = "I played my childhood game for a long time in 2011, of course I could not complete the tasks, I did not have a computer. I am playing now, I have been 250 levels in 8 months. When I enter TOP100, this is my only goal ... W1 idiom, AArif TR 260 LWL"
test['user_review'][5833] = "A review of the game Shakes and Fidget There they were again, the three problems: No gold, no experience and no idea how things will go now ^^ Don't worry, we will help you! You can find everything you need to know about shakes and fidget from us. All new features, opponents, dungeons and more ... The most important things to get started with Shakes and Fidget is all about gold, experience and other players knocking. You started to show everyone. No matter whether alone or with the help of your guild. How do you get gold and experience? Swing yourselves into the \ 'tavern \' and do adventures that are offered there. This gives you a lot of these things to develop your character and become stronger. If you run out of aluminum, you can buy up to 220 aluminum from the innkeeper for mushrooms. Initially only 200 until you are level 66 and gain access to the witch. Scrapbook: Scrapbook From level 10collects hundreds of items and improves your experience in adventures in the tavern. The dungeons: the dungeons from level 10, finish one dungeon after the other and show the monsters where the club hangs ^^ There are also delicious epic equipment items and a lot of experience points. The dungeon keys: the dungeon keys from level 10 in order to enter dungeons you need a dungeon key that opens them. You can find them now and then in the adventures of the tavern or the magic shop. How do you get gold and experience faster? The easiest way is with a mount from the stable. It’s best to grab the most expensive creature right away. The 25 mushrooms are well invested. The smaller mounts make no sense and there are more than enough mushrooms. You don't have to buy them from Shakes and Fidget :-) The mirror: The mirror from level 50 Collect all mirror parts to complete your mirror. This allows you to beat other players or enter a dungeon during an adventure. The Witch: The Witch from level 66 The witch enables you to enchant your weapons and armor items to further strengthen your character. The Tower: The Tower After 9 Dungeons As soon as you have finished all dungeons you are allowed to enter the tower. There you will find more opponents and the demon portal. In addition, three helpers will be made available to you for the battle in the tower. How do you get more experience and gold? You can also beat up other players in the \ 'arena \', but beware, these are usually a lot stronger than you. To avoid this, just trot over to the \ 'Ehrenhalle \' and look for easier victims. The toilet: the toilet from level 100 Everything you wash down the toilet here turns into objects that you can put on your three companions in the tower. When the pot is full, a rinse creates more epic items. Demon portal (dungeon): Demon portal (dungeon) after 9 dungeons Here you have to slaughter more demons :-) But they recover by 10% every day and represent a real challenge for every shakes and fidget gamers. Demon portal (guild): Demon portal (guild ) From level 99 here you fight with your guild members against powerful monsters. These recover by 10% every day. Every member is allowed to thrash this opponent once a day. Meeeeehr Gold Yes, yes, that's fine ... ^^ There is another way to get the delicious gold taler. If you have used up your \ 'ALU \' (thirst for adventure) in the tavern and are tired of thrashing others, just go to \ 'Stadtwache \'. You can get gold for doing nothing for up to 10 hours. However, you cannot do anything else then. Bugs? Sound? Graphic? Unfortunately there is a small bug in the appearance that your HP will fall or rise continuously (not fight relevant), I like the sounds very much, as well as the graphics because this game was brought from the browser to Steam. Joine for a great community and a professional curation program! As of: 11.11.2016"
test['user_review'][6408] = "Overall Score: 7.1 / 10 Quality 7.7/10 Satisfying quality for a casual Mt G Performance 6/10 Smooth overall; minor antialiasing issues(uneven edges); crashes may rarely occur Utility 8/10 Minor bugs (super rare); requires internet connection to save; Windows (7+) only Controls 9/10 Good; full pad support; turn phase time limit (only relevant in MP) a bit too short Gameplay 7.5/10 TBS fantasy CCG with high replayability Challenge 9/10 A complex, timeless strategic card game; (ranked) PVP duels; at times irrational AI Content 6/10 Loads of cards to collect and a constantlyupdated card pool; very basic story mode Diversity 8/10 Thousands of possible decks thanks to thehuge card pool; 4 (similar) game modes Immersion 7/10 Addictive CCG and deck creation system;challenging TBS-PVP (high skill range) Design 6.2/10 Beautiful artwork, but lame game design Visuals 9/10 Outstanding card artworks, but barely any animations; simple graphics and design Sound 6/10 Too monotone music, yet the devs at leastupdate it with every new set; SFX are OKTexts 8/10 Well written and localized cards, but story (mode) is disappointing: short & very basic Mapping 4/10 No levels / world just the card table; good overview, but no table customization at all Interface 4/10 Cheap, but functional menus; simplified turn phases compared to actual entry for Magic: The Gathering The story mode doesn't require using any own cards!"
test['user_review'][7304] = "Nice game. Recommend."
test['user_review'][7513] = "Early Access Review Well guys, here's the thing, I've been playing this game here since the time when it had no division, that is, when just survive and king of the kill were the same game. At that time, the game was extremely poorly optimized. I have a pc with an i5 3330 processor, a gtx 750 ti video card and 8gb of ram and with these configurations the game should run on ultra with a good slack, but that was not what happened. Not even in the low, with the textures and graphic qualities, he ran right. This game had constant drops of FPS. Most of the time, running at 100 fps, it dropped quickly to 20, 15, and that was straightforward, almost every minute of the game, making it impossible to play. Now in the division, the game was a little more optimized, but almost nothing at all. FPS drops are still very constant, regardless of the graphics quality that is left. In my view, it is still unplayable. I hope that someday, daybreak will decide to optimize this game for all users to be able to enjoy an acceptable graphic quality to play, especially those who have a pc with the recommended requirements to play in the largest possible graphic and that unfortunately cannot, due to the team's poor commitment to bringing a well-optimized game. Below is the first analysis I made of the game, positively, when I was eager to play it. After a few hours of playing, I came to do another analysis, negating. And now I'm doing the final analysis, but I won't take the first one I did, follow below if you want to see the first analysis I made of the game, a very good analysis Hue Hue BR:> Full 2015> Living in the northeast> Internet of 1 mega that only 500 kb> Bolado.jpeg> Buy the game in the Christmas promotion for 24 bucks> Still sad because it will take a century to download> Boot to download the game> 4.7gb game> Lowering to 40kb ~ 50kb> Estimated 3 days to complete the game> VERY TRISTE.exe> ​​2 days downloading> 3 days downloading> Few hours to finish the download> VERY HAPPY.rar> The download is over> Run the game> Create an account to be able to enter the game> Account created > Open the game> G29 error game> Pain in the heart.psd> Start again> Now game has entered> Very happy.bat> Poorly optimized game> Hitting hard but still don't give up playing> Start the battle game mode royale> Falling from parachutes> Fell> Two steps> One shot from the fifths of the hells> Killed me> Hate.png> I open it again> Bug Bug One more bug I can enter air in game mode> Zombie appears out of nowhere> Killed me> Hate. I enter again> I find a ♥♥♥♥ ing weapon with several ♥♥♥♥ ings ammunition> I enter the rogue pvp tomb> I kill two in a row> Happy pra ♥♥♥♥♥♥♥ . Now just survive"
test['user_review'][7535] = "Early Access Review Lots of bugs that virtually prevent gameplay. A man runs along the street, gets 3 shotguns (!) In the head, after which he can get -15hp. I do not mention the spontaneous car explosion, falling under the map etc."


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#r

In [139]:
train.drop([2777], inplace = True)

In [140]:
# remove pictograms
test['user_review'][72] = 'Good game. I do recommend'
test['user_review'][298] = 'Bad game.'
test['user_review'][300] = 'Good game.'
test['user_review'][2676] = 'Good game'
test['user_review'][5835] = 'Good game'
test['user_review'][5846] = 'Good game'
test['user_review'][7483] = 'Good game'
test['user_review'][7498] = "good game"
test['user_review'][7844] = 'nice game'


test['user_review'][4735] = 'Good game. I do recommend'
test['user_review'][7249] = 'Good game. I do recommend'
test['user_review'][3984] = 'Good game. I do recommend'


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#r

In [ ]:
# distribution_of_target = train.groupby(['title'])['user_suggestion'].mean().sort_values(ascending = False)
# distribution_of_target

# pp = train.groupby(['title', 'user_suggestion']).count()

# print(pp.to_string())

<h3> Clean text 

In [141]:
# Contractions

def clean(text):
    
    text = re.sub(" he's ", " he is ", text)
    text = re.sub(" there's ", " there is ", text)
    text = re.sub(" we're ", " we are ", text)
    text = re.sub("you'll", " you will ", text)

    text = re.sub(" that's ", " that is ", text)
    text = re.sub(" won't ", " will not ", text)
    text = re.sub(" they're ", " they are ", text)
    text = re.sub(" wan't ", " cannot ", text)
    text = re.sub(" wasn't ", " was not ", text)
    text = re.sub(" aren't ", " are not ", text)
    text = re.sub(" isn't ", " is not ", text)
    text = re.sub("haven't", " have not ", text)
    text = re.sub(" hasn't ", " has not ", text)
    text = re.sub(" there's ", " there is ", text)
    text = re.sub(" he's ", " he is ", text)
    text = re.sub(" it's ", " it is ", text)
    text = re.sub(" wou're ", " you are ", text)
    text = re.sub(" isnt ", " is not ", text)
    text = re.sub(" shouldn't ", " should not ", text)
    text = re.sub(" wouldn't ", " would not ", text)
    text = re.sub("i'm", " i am ", text)
    text = re.sub("i'm ", " i am ", text)
    text = re.sub(" im ", " i am ", text)
    text = re.sub(" youre ", " you are ", text)
    text = re.sub(" ive ", " i have ", text)
    text = re.sub(" cant ", " can not ", text)
    text = re.sub(" ive ", " i have ", text)
    text = re.sub(" dont ", " do not ", text)
    text = re.sub("doesnt", " does not ", text)
    text = re.sub(" thats ", " that is ", text)
    text = re.sub(" doesnt ", " does not ", text)
    text = re.sub(" don ‘ t ", " do not ", text) 
    text = re.sub(" i'll", " i will ", text)

    text = re.sub(" isn't ", " is not ", text)
    text = re.sub(" here's", " here is ", text)
    text = re.sub(" you've", " you have ", text)
    text = re.sub(" we're", " we are ", text)
    text = re.sub(" what's", " what is ", text)
    text = re.sub(" couldn't", "could not", text)
    text = re.sub(" we've", " we have ", text)
    text = re.sub(" who's", " who is ", text)
    text = re.sub(" y'all", " you all ", text)
    text = re.sub(" would've", " would have ", text)
    text = re.sub(" it'll", " it will ", text)
    text = re.sub(" we'll", " we will ", text)
    text = re.sub(" we've", " we have ", text)
    text = re.sub(" he'll", " he will ", text)
    text = re.sub(" y'all", " you all ", text)
    text = re.sub(" weren't", " were not ", text)
    text = re.sub(" didn't", " did not ", text)
    text = re.sub(" they'll", " they will ", text)
    text = re.sub(" they'd", " they would ", text)
    text = re.sub(" they've", " they have ", text)
    text = re.sub(" i'd", " i would ", text)
    text = re.sub(" should've", " should have ", text)
    text = re.sub(" where's", " where is ", text)
    text = re.sub(" we'd", " we would ", text)
    text = re.sub(" weren't", " were not ", text)
    text = re.sub(" they're", " they are ", text)
    text = re.sub(" let's", " let us ", text)
    text = re.sub(" it's", " it is ", text)
    text = re.sub(" can't", " cannot ", text)
    text = re.sub(" don't", " do not ", text)
    text = re.sub(" you're", " you are ", text)
    text = re.sub(" i've", " i have ", text)
    text = re.sub(" that's", " that is ", text)
    text = re.sub(" doesn't", " does not ", text)
    text = re.sub(" i'd", " i would ", text)
    text = re.sub(" didn't", " did not ", text)
    text = re.sub(" ain't", " am not ", text)
    text = re.sub(" you'll", " you will ", text)
    text = re.sub(" i've", " i have ", text)
    text = re.sub(" don't", " do not ", text)
    text = re.sub("i'll", " i will ", text)
    text = re.sub(" i'd", " i would ", text)
    text = re.sub(" let's", " let us ", text)
    text = re.sub(" you'd", " you would ", text)
    text = re.sub(" it's", " it is ", text)
    text = re.sub(" ain't", " am not ", text)
    text = re.sub(" could've", " could have ", text)
    text = re.sub("youve", " you have ", text)  
    text = re.sub("i'm", " i am ", text)   

            
#     Character entity references
    text = re.sub("&gt;", ">", text)
    text = re.sub("&lt;", "<", text)
    text = re.sub("&amp;", "&", text)
    
#     Strange words correction
    text = re.sub("gavno", "gavno", text)
    text = re.sub("gavno gavno gavno", "gavno gavno gavno", text)
    text = re.sub("gavno;", "gavno", text)
    text = re.sub("f*ck", "fuck", text)
    text = re.sub(" gg gg gg ", "good game", text)
    text = re.sub(" gg gg ", "good game", text)
    text = re.sub(" gg ", "good game", text)
#     text = re.sub("nope", "not", text)
#     text = re.sub("nope nope", "not not", text)
    text = re.sub("trashtrash", "trash trash", text)
    text = re.sub("deadtaken", "dead taken", text)
    text = re.sub("crashcrash", "crash crash", text)
    text = re.sub("reviewafte", "review afte", text)
    text = re.sub("reviewvery", "review very", text)
    text = re.sub("reviewdo", "review do", text)
    text = re.sub("reviewits", "review it", text)
    text = re.sub("reviewi've", "review", text)
    text = re.sub("deadtaken", "dead taken", text)
    text = re.sub("reviewas", "reviews", text)
    text = re.sub("reviewwhen", "review when", text)
    text = re.sub("deadtaken", "dead taken", text)
    text = re.sub("reviewi", "review", text)

    text = re.sub("reviewnot", "review not", text)
    text = re.sub("reviewfirst", "review first", text)
    text = re.sub("s s s", "sorry so sloppy", text)
    text = re.sub("reviewthis", "review this", text)
    text = re.sub("reviewgreat", "review great", text)
    text = re.sub("freeearly", "free early", text)
    text = re.sub("reviewthe", "review the", text)
    text = re.sub("reviewa", "review", text)
    text = re.sub(" si ", "simple", text)
    text = re.sub(" ame ", " time ", text)
    text = re.sub(" ames ", " times ", text)
    text = re.sub(" si ample ", "simple", text)
    text = re.sub(" ti ame ", " time ", text)
    text = re.sub(" ti ", " time ", text)
    text = re.sub(" ti ames ", " times ", text)
#     text = re.sub(" fck ", " ", text)
    text = re.sub("â", "", text)
    text = re.sub(" si amply ", "simply", text)
    text = re.sub("reviewso", "review so", text)
    text = re.sub("reviewif", "review if", text)
    text = re.sub(" si amulator ", "simulato", text)
#     text = re.sub("bafufuck", "fuck", text)
#     text = re.sub("defufuck", "fuck", text)
#     text = re.sub("attafufuckforwardstrong", "fuck forward strong", text)
#     text = re.sub("sufufuck", "fuck", text)
#     text = re.sub("clifufuck", "fuck", text)
#     text = re.sub("hafufuckers", "fuck", text) 
    text = re.sub(" sI ample", "simple", text)
#     text = re.sub("o", "o", text)
#     text = re.sub(" hmm ", "", text)
#     text = re.sub("lofufufucked", "", text)
    text = re.sub(" ti ames ", "time", text)
#     text = re.sub("lofucked", "", text)
    text = re.sub(" oh ", " ", text)
    text = re.sub(" its ", " it is ", text)
    text = re.sub(" you'll ", "you will", text)
#     text = re.sub("computer some", " computer some ", text)
    text = re.sub("isnot", " is not ", text)
    text = re.sub("atleast", "at least", text)
    text = re.sub("lets ", "let ", text)
    text = re.sub("'s", " ", text)
    text = re.sub("ensues", " ensure ", text)
#     text = re.sub("™¥", " ", text)  
#     text = re.sub("° ͜ʖ ͡°", " ", text) 
#     text = re.sub("° ͡° ͜ʖ", " ", text)       
#     text = re.sub("ʖ ͡° ͡°", " ", text)
#     text = re.sub("☉ ͜ʖ ͡☉", " ", text)
#     text = re.sub("☉ ͡☉ ͜ʖ", " ", text)
#     text = re.sub("ʖ ͡☉ ͡☉", " ", text)
#     text = re.sub(" ͡ ͡ ", " ", text)     
#     text = re.sub(" ͡ ͡ ͡ ", " ", text) 
#     text = re.sub(" ͡ ", " ", text)   
#     text = re.sub("♥", " love ", text)
#     text = re.sub(" bafuck ", " ", text)  
#     text = re.sub(" bafufuck ", " ", text)
#     text = re.sub(" bafufufuck ", " ", text)
#     text = re.sub(" bafufufufuck ", " ", text)
#     text = re.sub(" clifufufufuck ", " ", text)
#     text = re.sub(" clifufufufufuck ", " ", text)
#     text = re.sub(" clifufufufufufuck ", " ", text)
#     text = re.sub(" clifufufufufufufuck ", " ", text)
#     text = re.sub(" clifufufufufufufufuck ", " ", text)
#     text = re.sub(" clifufufufufufufufufuck ", " ", text)
#     text = re.sub(" clifuffuufufufufufufufuck ", " ", text)
#     text = re.sub(" clifufufufufufufuck ", " ", text)
#     text = re.sub(" sufufufufuck ", " ", text)
#     text = re.sub(" sufufufufufuck ", " ", text)
#     text = re.sub(" sufufufufufufuck ", " ", text)
#     text = re.sub(" sufufufufufufufuck ", " ", text)
#     text = re.sub(" sufufufufufufufufuck ", " ", text)
#     text = re.sub(" sufufufufufufufufufuck ", " ", text)
#     text = re.sub(" sufufufufufufufufufufuck ", " ", text)
#     text = re.sub(" sufufufufufufufufufufufuck ", " ", text)
#     text = re.sub(" attafufufufuck ", " ", text)
#     text = re.sub(" attafufufufufuck ", " ", text)
#     text = re.sub(" attafufufufufufuck ", " ", text)
#     text = re.sub(" attafufufufufufufuck ", " ", text)
#     text = re.sub(" attafufufufufufufufuck ", " ", text)
#     text = re.sub(" attafufufufufufufufufuck ", " ", text)
#     text = re.sub(" attafufufufufufufufufufuck ", " ", text)
#     text = re.sub(" attafufufufufufufufufufufuck ", " ", text)
    
#     text = re.sub(" hafufufufuckers ", " ", text)
#     text = re.sub(" hafufufufufufufuckers ", " ", text)
#     text = re.sub(" hafufufufufufufufuckers ", " ", text)
#     text = re.sub(" hafufufufufufufufufuckers ", " ", text)
#     text = re.sub(" hafufufufufufufuckers ", " ", text)    
#     text = re.sub(" hafufufufufufufufuckers ", " ", text)
#     text = re.sub(" hafufufufufufufufuckers ", " ", text)

    
#     text = re.sub(" fufuck ", " ", text)
#     text = re.sub(" fufufuck ", " ", text)

#     text = re.sub(" fufufufuck ", " ", text)
#     text = re.sub(" fufufufufufufuck ", " ", text)
#     text = re.sub(" fufufufufufufufuck ", " ", text)
#     text = re.sub(" fufufufufufufufufuck ", " ", text)
#     text = re.sub(" fufufufufufufuck ", " ", text)

    
    
    text = re.sub(" reviewwell ", " review well ", text)
    text = re.sub(" reviewf ", " review ", text)
#     text = re.sub(" ͜ʖ", " ", text)
    
    text = re.sub("o o o o", " ", text)
#     text = re.sub(" dufufufufucks ", " ", text)
#     text = re.sub(" dufufufufufufufufucks ", " ", text)  
#     text = re.sub(" yu gi oh ", " ", text)
    text = re.sub(" realm ", " real ", text)
#     text = re.sub(" rakovina ", " ", text)
#     text = re.sub(" rakovina rakovina ", " ", text)
#     text = re.sub(" rakovina rakovina rakovina ", " ", text)
#     text = re.sub(" rakovina rakovina rakovina rakovina ", " ", text)
#     text = re.sub(" rakovina rakovina rakovina rakovina rakovina ", " ", text)
#     text = re.sub(" rakovina rakovina rakovina rakovina rakovina rakovina ", " ", text)
#     text = re.sub(" rakovina rakovina rakovina rakovina rakovina rakovina rakovina ", " ", text)

#     text = re.sub("fufufufufufuck", " fuck ", text)  
#     text = re.sub("attafufufufufufufuck", " fuck ", text)
#     text = re.sub("[0-9]", "", text)
    
    
#     punctuations = '#$%&\'()+,-/:;<=>@[\\]^_`{|}~¯( ͡° ͜ʖ ͡°) ͜ʖ° ͜ʖ ͡°ʖ ͡☉ ͡☉ ͡ ͡ ͡  ͡ '
#     for p in punctuations:
#         text = text.replace(p, ' ')
    
    text = re.sub(' +', ' ', text)
#     text = re.sub(" Oh ", "", text)

    return text

train['user_review'] = train['user_review'].apply(lambda x: clean(x))
test['user_review'] = test['user_review'].apply(lambda x: clean(x))

In [142]:
def remove_punctuation(text):    
    punctuations = '¯™¥-♥#$%&:;\()+-<=>@[\\]^_`{|}~/¯( ͡° ͜ʖ ͡°) ͜ʖ° ͜ʖ ͡°ʖ ͡☉ ͡☉ ͡ ͡ ͡  ͡ ▌░╔╗║╔═╗'
    for p in punctuations:
        text = text.replace(p, ' ')
    text = re.sub(' +', ' ', text)
    return text
        
train['user_review'] = train['user_review'].apply(lambda x: remove_punctuation(x)) 
test['user_review'] = test['user_review'].apply(lambda x: remove_punctuation(x))    

In [143]:
# remove chinese characters
train['user_review'] = train['user_review'].apply(lambda x: re.sub('[^\x00-\x7F]+', ' ', x))
test['user_review'] = test['user_review'].apply(lambda x: re.sub('[^\x00-\x7F]+', ' ', x))

In [144]:
def remove_emoji(text):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags = re.UNICODE)
    return emoji_pattern.sub(r'', text)

train['user_review'] = train['user_review'].apply(lambda x: remove_emoji(x))
test['user_review'] = test['user_review'].apply(lambda x: remove_emoji(x))

In [145]:
# remove spaces
train['user_review'] = train['user_review'].apply(lambda x: re.sub(' +', ' ', x))
test['user_review'] = test['user_review'].apply(lambda x: re.sub(' +', ' ', x))

In [146]:
train[train['user_review'].str.contains(' f*ck ')].head()

,review_id,title,year,user_review,user_suggestion
0,1,Spooky's Jump Scare Mansion,2016.0,I'm scared and hearing creepy voices. So I'll ...,1
4,5,Spooky's Jump Scare Mansion,2015.0,Not many games have the cute tag right next to...,1
8,9,Spooky's Jump Scare Mansion,2015.0,"Somewhere between light hearted, happy parody ...",0
9,10,Spooky's Jump Scare Mansion,2015.0,This game with it is cute little out of the wa...,1
17,18,Spooky's Jump Scare Mansion,2015.0,It exactly what you think it is. Progress thro...,1


In [147]:
train['user_review'][0]

"I'm scared and hearing creepy voices. So I'll pause for a moment and write a review while I wait for my heart beat to return to at least somewhat calmer times. This game is adorable and creepy like my happy tree friends but with the graphics sceme of my childhood but more bubble and 'clean' . Hello 1990 . What charactes there are that is not trying to kill me were likable and a bit odd. I did do a few noob things though, such as Oh look a class room full of ghosts from dead children, let shine my flashlight on them and stand there staring at them.. Or, hmm creepy music, I'll turn around and see if I can see what is chasing me. Never before in a game have I been this afraid of finding a lofucked door."

In [118]:
train.head(50)

,review_id,title,year,user_review,user_suggestion
0,1,Spooky's Jump Scare Mansion,2016.0,I'm scared and hearing creepy voices. So I'll ...,1
1,2,Spooky's Jump Scare Mansion,2016.0,"Best game, more better than Sam Pepper You Tub...",1
2,3,Spooky's Jump Scare Mansion,2016.0,"A littly iffy on the controls, but once you kn...",1
3,4,Spooky's Jump Scare Mansion,2015.0,"Great game, fun and colorful and all that. A s...",1
4,5,Spooky's Jump Scare Mansion,2015.0,Not many games have the cute tag right next to...,1
5,6,Spooky's Jump Scare Mansion,2015.0,"Early Access Review It pretty cute at first, b...",1
6,7,Spooky's Jump Scare Mansion,2017.0,Great game. it is a cute little horror game th...,1
7,8,Spooky's Jump Scare Mansion,2015.0,Spooky Jump Scare Mansion is a Free Retro maze...,1
8,9,Spooky's Jump Scare Mansion,2015.0,"Somewhere between light hearted, happy parody ...",0
9,10,Spooky's Jump Scare Mansion,2015.0,This game with it is cute little out of the wa...,1


In [113]:
test['user_review'][7326]

"This game, this game... I have not been playing for that long of a time, but damn I've enjoyed it quite a lot! Personally it is been a lot more fun with a group of friends, but it also can be a lot of fun with the hidden quests, the endless titles, the epic Eidolon pets, fun interactive quests, random world bosses, making a guild. It just...14 dance emotes, plus a kiss emote ... I like."

In [111]:
np.where(test['user_review'].apply(lambda x: x == ' '))


(array([], dtype=int64),)

In [114]:
test.shape

(8045, 4)

In [ ]:
train.to_csv('train_cleaned_v2.csv', index = False)
test.to_csv('test_cleaned_v2.csv', index = False)

<h3> Roberta

In [ ]:
import tensorflow as tf

In [ ]:
!pip install ktrain

In [ ]:
import ktrain
from ktrain import text

In [ ]:
x_train, x_valid, y_train, y_valid = train_test_split(train['user_review'].values, train['user_suggestion'].values, test_size = 0.2)

In [ ]:
MODEL_NAME = 'roberta_large'

In [ ]:
t = text.Transformer(MODEL_NAME, maxlen = 220, class_names = [0, 1])
trn = t.preprocess_train(x_train, y_train)
val = t.preprocess_train(x_valid, y_valid)
model = t.get_classifier()
learner = ktrain.get_learner(model, train_data = trn, val_data = val, batch_size = 8)

In [ ]:
learner = lr_find(show_plot = True, max_epochs = 2)

In [ ]:
learner.fit_onecycle(2e-5, 2)

In [ ]:
learner.validate(class_names = t.get_classes())

In [ ]:
preds = predictor.predict(test.user_review.values)
submission['user_suggestion'] = preds

In [ ]:
submission.to_csv('sub_1.csv', index = False)

<h3> N-gramms

In [127]:
%%time
def generate_ngrams(user_review, n_gram = 1):
    token = [token for token in user_review.lower().split(' ') if token != '' if token not in STOPWORDS]
    ngrams = zip(*[token[i:] for i in range(n_gram)])
    return [' '.join(ngram) for ngram in ngrams]

N = 100

recommend_review = train['user_suggestion'] == 1

# Unigrams
recommend_unigrams = defaultdict(int)
nonrecommend_unigrams = defaultdict(int)

for review in train[recommend_review]['user_review']:
    for word in generate_ngrams(review):
        recommend_unigrams[word] += 1
        
for review in train[~recommend_review]['user_review']:
    for word in generate_ngrams(review):
        nonrecommend_unigrams[word] += 1
        
df_recommend_unigrams = pd.DataFrame(sorted(recommend_unigrams.items(), key=lambda x: x[1])[::-1])
df_nonrecommend_unigrams = pd.DataFrame(sorted(nonrecommend_unigrams.items(), key=lambda x: x[1])[::-1])

# Bigrams
recommend_bigrams = defaultdict(int)
nonrecommend_bigrams = defaultdict(int)

for review in train[recommend_review]['user_review']:
    for word in generate_ngrams(review, n_gram=2):
        recommend_bigrams[word] += 1
        
for review in train[~recommend_review]['user_review']:
    for word in generate_ngrams(review, n_gram=2):
        nonrecommend_bigrams[word] += 1
        
df_recommend_bigrams = pd.DataFrame(sorted(recommend_bigrams.items(), key=lambda x: x[1])[::-1])
df_nonrecommend_bigrams = pd.DataFrame(sorted(nonrecommend_bigrams.items(), key=lambda x: x[1])[::-1])

# Trigrams
recommend_trigrams = defaultdict(int)
nonrecommend_trigrams = defaultdict(int)

for review in train[recommend_review]['user_review']:
    for word in generate_ngrams(review, n_gram=3):
        recommend_trigrams[word] += 1
        
for review in train[~recommend_review]['user_review']:
    for word in generate_ngrams(review, n_gram=3):
        nonrecommend_trigrams[word] += 1
        
df_recommend_trigrams = pd.DataFrame(sorted(recommend_trigrams.items(), key=lambda x: x[1])[::-1])
df_nonrecommend_trigrams = pd.DataFrame(sorted(nonrecommend_trigrams.items(), key=lambda x: x[1])[::-1])

# Fourgrams
recommend_fourgrams = defaultdict(int)
nonrecommend_fourgrams = defaultdict(int)

for review in train[recommend_review]['user_review']:
    for word in generate_ngrams(review, n_gram=4):
        recommend_fourgrams[word] += 1
        
for review in train[~recommend_review]['user_review']:
    for word in generate_ngrams(review, n_gram=4):
        nonrecommend_fourgrams[word] += 1
        
df_recommend_fourgrams = pd.DataFrame(sorted(recommend_fourgrams.items(), key=lambda x: x[1])[::-1])
df_nonrecommend_fourgrams= pd.DataFrame(sorted(nonrecommend_fourgrams.items(), key=lambda x: x[1])[::-1])

CPU times: user 10.9 s, sys: 352 ms, total: 11.2 s
Wall time: 11.2 s


In [128]:
df_nonrecommend_trigrams.head(60)

,0,1
0,early access review,2289
1,rakovina rakovina rakovina,852
2,virus virus virus,388
3,access review game,347
4,"active, game session",299
5,"session active, game",299
6,"game session active,",299
7,fix fix fix,283
8,"pay, pay, pay,",234
9,product received free,189


<h3> Simple classifier Log Reg with tf idf

In [ ]:
# import nltk
# from nltk.tokenize import sent_tokenize,word_tokenize
# from nltk.stem import WordNetLemmatizer 
# from nltk.corpus import stopwords 
# from sklearn.feature_extraction.text import TfidfVectorizer

# import re
# re_tok = re.compile(f'([{string.punctuation}“”¨«»®´·º½¾¿¡§£₤‘’])')
# def tokenize(s): 
#     return re_tok.sub(r' \1 ', s).split()

# # vectorizer = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}')

# # vectorizer = TfidfVectorizer(min_df=3,  max_features=None, 
# #             strip_accents='unicode', analyzer='word',token_pattern=r'\w{1,}',
# #             ngram_range=(1, 3), use_idf=1,smooth_idf=1,sublinear_tf=1,
# #             stop_words = 'english')

# vectorizer = TfidfVectorizer(min_df=0.0, max_df=1.0, ngram_range=(1,2),sublinear_tf=True)

# # vectorizer = TfidfVectorizer(tokenizer=tokenize)
# train_vec = vectorizer.fit_transform(train.user_review.values)



In [ ]:
# test_vec = vectorizer.transform(test.user_review.values)

In [ ]:
# from sklearn.linear_model import LogisticRegression
# from sklearn.linear_model import RidgeClassifier 
# lr = LogisticRegression(C=5, dual=True, solver='liblinear', max_iter = 10000)
# rr = RidgeClassifier()

In [ ]:
# scores = cross_val_score(lr, train_vec, train.user_suggestion.values, cv = 5, scoring = "f1")
# print(np.mean(scores))

# # scores = cross_val_score(nb, train_vec, train.user_suggestion.values, cv = 5, scoring = "f1")
# # print(np.mean(scores))

In [ ]:
# svm = SVC()
# # scores = cross_val_score(svm, train_vec, data[:train.shape[0]].user_suggestion.values, cv = 3, scoring = "f1")
# # print(np.mean(scores))

In [ ]:
import tensorflow as tf

# Get the GPU device name.
device_name = tf.test.gpu_device_name()

# The device name should look like the following:
if device_name == '/device:GPU:0':
    print('Found GPU at: {}'.format(device_name))
else:
    raise SystemError('GPU device not found')

In [ ]:
import torch

# If there's a GPU available...
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")